In [162]:
import pypsa
import numpy as np
import pandas as pd
import os
import math

In [163]:
period_start = "2018-01-01"
period_end = "2018-12-31"
time_start = '{} 00:00:00'.format(period_start)
time_end = '{} 23:00:00'.format(period_end)
timespan = pd.date_range(start=time_start, end=time_end, freq='H')

In [164]:
#directory = 'D:\Python\PyPSA/Luca/nodal_zonal_model/hydro/hpc_results/last_results/hour0/0.5/bids/20/'
# directory = 'D:\Python\PyPSA/Luca/nodal_zonal_model/hydro/hpc_results/last_results/hour0/{}/{}/{}/'
directory = 'D:\Python\PyPSA/Luca/nodal_zonal_model/hydro/hpc_results/last_results/hour0/zonal/zonal_1024_cost2018_results_1y_min30percent.nc'
n = pypsa.Network(directory)

INFO:pypsa.io:Imported network zonal_1024_cost2018_results_1y_min30percent.nc has buses, carriers, generators, links, loads, storage_units


In [131]:
# # path_network = os.path.join(path,'network_merged.nc')
# path_network = 'D:\Python\PyPSA/Luca/nodal_zonal_model/hydro/hpc_results/last_results/hour0/0.5/bids/20/network_merged.nc'
# n = pypsa.Network(path_network)

INFO:pypsa.io:Imported network network_merged.nc has buses, carriers, generators, lines, links, loads, storage_units


In [165]:
# total capacity of storage units (max_hours*p_nom)
cap = n.storage_units.p_nom*n.storage_units.max_hours
# potential to close the gap
# i.e. inflow in hour 0 for hydro and p_nom*1hour (including efficiency) for PHS
close = dict()
for su in n.storage_units.index:
    if su.__contains__('hydro'):
        close[su] = n.storage_units_t.inflow.loc[timespan[0],su]
    elif su.__contains__('PHS'):
        close[su] = n.storage_units.p_nom[su]*n.storage_units.efficiency_store[su]

In [133]:
penalty_list = [17, 19,20,27,28,32, 35]
penalty_unit = dict()
penalty_unit[17] = 1000
penalty_unit[19] = 10
penalty_unit[20] = 0
penalty_unit[27] = 10
penalty_unit[28] = 0
penalty_unit[32] = 0
penalty_unit[35] = 10
penalty_sum = dict()
penalty_sum[17] = 1000
penalty_sum[19] = 1000
penalty_sum[20] = 1000
penalty_sum[27] = 10
penalty_sum[28] = 10
penalty_sum[32] = 0
penalty_sum[35] = 40

In [134]:
bids_list = [30, 40, 50, 60, 70]#['zonal_shadow', 20, 30, 40, 50, 60, 70]

In [166]:
# for later
# to consolidate results
out = dict()

# directory = 'D:\Python\PyPSA/Luca/nodal_zonal_model/hydro/hpc_results/last_results/hour0/{}/{}/{}/'
# transmission_cap = 0.5
# bids_or_penalty = 'bids'
# combi = 20

In [168]:
# if bids_or_penalty == 'bids':
#     combi_list = bids_list
# elif bids_or_penalty == 'penalty':
#     combi_list = penalty_list
# # for combi in combi_list:
#     dir_combi = directory.format(transmission_cap,bids_or_penalty,combi)
#     # to consolidate results
outputs = []
    # # add info of penalty or bid to outputs
    # outputs.append(bids_or_penalty)

    # read soc data
df = n.storage_units_t.state_of_charge
    # path = os.path.join(dir_combi,'soc_nodal_1y.csv')
    # df = pd.read_csv(path)
    # df.snapshot = pd.to_datetime(df.snapshot)
    # df = df.set_index('snapshot')
    # difference where negative values are corrected for close the gap potential from pumping and inflow
diff_corrected = (df.loc[timespan[-1]]-df.loc[timespan[0]])
for su in diff_corrected.index:
    if diff_corrected[su]<0: # if the discrepancy is negative take into account possible inflows and pumping up to close this gap
        diff_corrected[su] = diff_corrected[su] + close[su]
        if diff_corrected[su]>0: # if inflow and pumping are not only closing the gap but can make difference positive, set it to 0, not to have too nice results
            diff_corrected[su]=0
# absolute difference corrected
outputs.append(diff_corrected.sum())
# mean absolute difference pos and neg
outputs.append(diff_corrected.mean())
# mean rel difference wrt hour 1 (if hour 1 soc is 0 need to ignore these inf values)
outputs.append(((diff_corrected/df.loc[timespan[0]])[np.isfinite(diff_corrected/df.loc[timespan[0]])]).mean())
    ### other than soc
    # computational time
    # read from text file the run times
    # text_path = os.path.join(dir_combi,'info.txt')
    # with open(text_path) as f:
    #     lines = f.readlines()
    # for i,line in enumerate(lines):
    #     if 'finish' in line:
    #         time = float(lines[i+1].strip('\n'))
    # # append run time for main optimization to outputs in s and hours
    # outputs.append(time)
    # outputs.append(time/60/60)
ofv = n.objective
    # # read ofv 'corrected'
    # path_ofv = os.path.join(dir_combi,'objective_value_sum_1y.txt')
    # with open(path_ofv) as f:
    #     lines = f.readlines()
    #     ofv= float(lines[0].strip('\n'))
outputs.append(ofv)

    # # more results only if optimization was feasible
    # if ofv == np.inf:
    #     pass
    # else:
    #     path_n = os.path.join(dir_combi, 'network_merged.nc')
    #     n = pypsa.Network(path_n)
    #     ### Load shedding
ind_load = [gen for gen in n.generators.index if gen.__contains__('load')]
load_shed_sum = n.generators_t.p.loc[timespan,ind_load].sum().sum()
outputs.append(load_shed_sum)
load_shed_share = load_shed_sum/(n.loads_t.p.loc[timespan].sum().sum())
outputs.append(load_shed_share)

        # ### get the number of congestions
        # lines_and_hours = len(timespan)*len(n.lines.index)
        # # total number of times and lines when there is no congestion
        # no_congestions = n.lines_t.p0[abs(n.lines_t.p0)/(n.lines.s_nom*n.lines.s_max_pu)==1].loc[timespan,:].isnull().sum().sum()
        # congestions = lines_and_hours-no_congestions
        # # kind of share of congestion event over the total possible number
        # congestions_per = congestions/lines_and_hours
        # # congestion hours in every hour
        # congestion_hours_no = n.lines_t.p0[abs(n.lines_t.p0)/(n.lines.s_nom*n.lines.s_max_pu)==1].loc[timespan,:].isnull().sum(axis=1)
        # congestion_hours = len(n.lines.index) - congestion_hours_no
        # # this is the mean number of congested lines in every hour
        # congestion_hours_mean = congestion_hours.mean()

In [178]:
# congestions in links
links_and_hours = len(timespan)*len(n.links.index)
no_pos = n.links_t.p0[(n.links_t.p0)/(n.links.p_nom*n.links.p_max_pu)==1].loc[timespan,:].isnull().sum().sum()
no_neg = n.links_t.p0[(n.links_t.p0)/(n.links.p_nom*n.links.p_min_pu)==1].loc[timespan,:].isnull().sum().sum()
# n.links.p_min_pu # is negative but also p0 is negative when flow in opposite direction
congestions_links = (links_and_hours - no_pos) + (links_and_hours - no_neg)
congestions_links_per = congestions_links/links_and_hours
congestions_links_per

0.6280167903348554

In [ ]:
        ### system congestion from paper in energy
        # get average LMP in every hour
lmp_avg = n.buses_t.marginal_price.loc[timespan,:].mean(axis=1)
# sc_0 =  math.sqrt(1/len(n.buses.index)*(sum([(lmp_avg[year[0]] - n.buses_t.marginal_price.loc[year[0],bus])**2 for bus in n.buses.index])))
sc_t = [math.sqrt(1/len(n.buses.index)*(sum([(lmp_avg[t] - n.buses_t.marginal_price.loc[t,bus])**2 for bus in n.buses.index]))) for t in timespan]
sc_avg = sum(sc_t)/len(sc_t)
        # # add to results
        # outputs.append(congestions_per)
        # outputs.append(congestion_hours_mean)
outputs.append(sc_avg)
    # assign results to dictionary
out['zone'] = outputs

In [169]:
# the index of dataframe will be
ind = ['penalty/bid','abs diff corrected', 'mean abs diff','mean rel diff wrt hour 1','run time in s', 'run time in h','ofv in EUR','load shedding total in MWh', 'load shedding share of total demand', 'congestion share of total events', 'average no of congested lines per hour', 'average system congestion']
ind_z = ['abs diff corrected', 'mean abs diff','mean rel diff wrt hour 1','ofv in EUR','load shedding total in MWh', 'load shedding share of total demand', 'average system congestion']

In [119]:
# table2 = pd.DataFrame.from_dict(out,orient='index')
# table2 = table2.transpose()
# table2.columns = ind

ValueError: Length mismatch: Expected axis has 7 elements, new values have 12 elements

In [170]:
table = pd.DataFrame.from_dict(out,orient='index')
table.columns = ind_z

In [127]:
# calculate the difference in ofv from the smallest value
table['ofv diff to smallest in Mio EUR'] = (table['ofv in EUR']-table['ofv in EUR'].min())/1e6
# calculate also th epercentage difference
table['ofv diff wrt smallest'] = (table['ofv in EUR']-table['ofv in EUR'].min())/table['ofv in EUR'].min()
table

,penalty/bid,abs diff corrected,mean abs diff,mean rel diff wrt hour 1,run time in s,run time in h,ofv in EUR,load shedding total in MWh,load shedding share of total demand,congestion share of total events,average no of congested lines per hour,average system congestion,ofv diff to smallest in Mio EUR,ofv diff wrt smallest
zonal_shadow,bids,4.013991e+01,1.013634e-01,1.749195e-06,23385.53,6.495981,2.635893e+11,1.924431e+07,0.005946,0.029374,48.819977,1123.433002,2161.809271,0.008269
20,bids,-7.983764e-09,-2.016102e-11,-3.043078e-17,23661.55,6.572653,2.656342e+11,1.944426e+07,0.006008,0.029484,49.002740,1133.562292,4206.790278,0.016092
30,bids,1.829628e+02,4.620273e-01,7.917649e-06,17840.12,4.955589,inf,NaN,NaN,NaN,NaN,NaN,inf,inf
40,bids,-7.591595e-09,-1.917069e-11,-3.026724e-17,20205.52,5.612644,2.635348e+11,1.923932e+07,0.005944,0.029339,48.761187,1123.559604,2107.323043,0.008061
50,bids,2.508911e+02,6.335634e-01,1.081989e-05,21072.45,5.853458,inf,NaN,NaN,NaN,NaN,NaN,inf,inf
60,bids,1.915970e+03,4.838309e+00,8.262819e-05,19498.92,5.416367,inf,NaN,NaN,NaN,NaN,NaN,inf,inf
70,bids,1.180086e+02,2.980015e-01,5.089243e-06,23973.64,6.659344,2.614274e+11,1.903055e+07,0.005880,0.029480,48.994977,1130.142020,0.000000,0.000000


In [171]:
save_table = 'D:\Python\PyPSA/Luca/nodal_zonal_model/hydro/hpc_results/last_results/hour0/zonal_results.csv'
table.to_csv(save_table)

In [160]:
out = {}
out[(1,'test')] = 3
table = pd.DataFrame.from_dict(out,orient='index')
table

,0
"(1, test)",3
